# PIV Flags: Data Quality Management

**Learning Objectives**
- Understand the purpose and importance of flags in PIV data
- Learn how to create and use flag schemes in pivmetalib
- Implement different flag types (enumerated, bitwise)
- Apply flags to PIV datasets for quality control

**Prerequisites**
- Basic understanding of PIV data processing
- Familiarity with Python and pivmetalib basics
- Complete [Getting Started](GettingStarted.ipynb) tutorial first

**Estimated Time**: 20-30 minutes

---

## Introduction

PIV (Particle Image Velocimetry) evaluations typically use **flags** to define the status of each velocity vector. Flags are essential for:


📊 **Data Filtering**: Selecting only high-quality measurements
🔬 **Processing Tracking**: Recording which algorithms modified each vector
📈 **Statistics**: Computing metrics

The pivmetalib library provides a comprehensive flag system:

## Setup and Imports

In [1]:
# Additional imports for examples
import numpy as np

# Core imports for flag management
from pivmetalib.pivmeta import FlagScheme, Flag, BitwiseFlagScheme, EnumeratedFlagScheme

## Basic Flag Concepts

### What is a Flag?

A flag is a marker that indicates the status or quality of a PIV vector. Common flag types include:

- **Good**: Valid, reliable velocity measurement
- **Bad**: Invalid or unreliable measurement
- **Interpolated**: Value filled in by interpolation
- **Replaced**: Value replaced by algorithm
- **Edge**: Vector at image boundary

### Flag Schemes

A **FlagScheme** defines the complete set of possible flags and their meanings. It defines the flags (`allowedFlag`) and the `usesFlagSchemeType` used:

In [2]:
# Create a simple enumerated flag scheme
simple_scheme = FlagScheme(
    label="Basic PIV Quality Flags",
    description="Simple quality indicators for PIV vectors",
    usesFlagSchemeType=EnumeratedFlagScheme(id="https://example/enumeratedFlagSchemeType"),
    allowedFlag=[
        Flag(
            mask=1,
            label="Active",
            description="Valid and reliable velocity measurement",
            meaning="active"
        ),
        Flag(
            mask=2,
            label="Filtered",
            description="Value filterded by interpolation",
            meaning="filtered"
        ),
        Flag(
            value=3,
            label="Interpolated",
            description="Value interpolated by algorithm",
            mask="32",
            meaning="interpolated"
        )
    ]
)

print("Created simple flag scheme with", len(simple_scheme.allowedFlag), "flags")
print(simple_scheme.serialize("ttl"))

Created simple flag scheme with 3 flags
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix piv: <https://matthiasprobst.github.io/pivmeta#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<https://example/enumeratedFlagSchemeType> a piv:EnumeratedFlagScheme .

[] a piv:FlagScheme ;
    rdfs:label "Basic PIV Quality Flags" ;
    dcterms:description "Simple quality indicators for PIV vectors" ;
    piv:allowedFlag [ a piv:Flag ;
            rdfs:label "Filtered" ;
            dcterms:description "Value filterded by interpolation" ;
            piv:mask 2 ;
            piv:meaning "filtered" ],
        [ a piv:Flag ;
            rdfs:label "Interpolated" ;
            dcterms:description "Value interpolated by algorithm" ;
            piv:mask 32 ;
            piv:meaning "interpolated" ],
        [ a piv:Flag ;
            rdfs:label "Active" ;
            dcterms:description "Valid and reliable velocity measurement" ;
   

## Advanced Flag Schemes

### Bitwise Flags

Bitwise flags allow combining multiple states in a single integer value. This is useful when vectors can have multiple quality issues simultaneously:

In [3]:
# Create a bitwise flag scheme
bitwise_scheme = FlagScheme(
    label="Advanced PIV Quality Flags",
    description="Bitwise combination of quality indicators",
    usesFlagSchemeType=BitwiseFlagScheme(id="https://example/enumeratedFlagSchemeType"),
    allowedFlag=[
        Flag(
            mask=1,
            label="Active",
            description="Valid and reliable velocity measurement",
            meaning="active"
        ),
        Flag(
            mask=8,
            label="Disabled",
            description="Value disabled",
            meaning="disabled"
        ),
        Flag(
            mask=16,
            label="Filtered",
            description="Value filterded by interpolation",
            meaning="filtered"
        ),
        Flag(
            label="Interpolated",
            description="Value interpolated by algorithm",
            mask=32,
            meaning="interpolated"
        ),
        Flag(
            label="Replaced",
            description="Value replaced by algorithm",
            mask=64,
            meaning="replaced"
        )
    ]
)

# Example: A vector with low correlation AND at edge = 1 | 4 = 5
combined_flag = 1 | 16
print(f"Combined flag value: {combined_flag}")

# Check which conditions are present
if combined_flag & 1:  # Low correlation
    print("active fector")
if combined_flag & 16:  # Edge vector
    print("filtered")


# Get all active flags for a value
def get_active_flags(value, scheme):
    active = []
    for flag in scheme.allowedFlag:
        if value & flag.mask:
            active.append(flag)
    return active


active_flags = get_active_flags(combined_flag, bitwise_scheme)
print(f"Active flags: {[str(f.label) for f in active_flags]}")

Combined flag value: 17
active fector
filtered
Active flags: ['Active', 'Filtered']


## Practical Example: Applying Flags to PIV Data

Let's create a practical example of applying flags to synthetic PIV data:

In [4]:
# Generate synthetic PIV data
np.random.seed(42)
height, width = 50, 100

# Create velocity fields
u = np.random.normal(0, 1, (height, width))  # x-velocity
v = np.random.normal(0, 1, (height, width))  # y-velocity

# Create flag array (start with all good)
flags = np.zeros((height, width), dtype=int)

# Simulate various quality issues
# High velocity outliers
high_vel_mask = (np.abs(u) > 3) | (np.abs(v) > 3)
flags[high_vel_mask] = 32  # Interpolated

# Edge vectors
flags[0, :] = 64  # Replaced
flags[-1, :] = 64
flags[:, 0] = 64
flags[:, -1] = 64

# Random bad vectors
bad_indices = np.random.choice(height * width, size=int(0.05 * height * width), replace=False)
flags.flat[bad_indices] = 2  # Bad

print(f"Generated PIV data: {u.shape}")
print(f"Flag distribution:")
unique, counts = np.unique(flags, return_counts=True)

for val, count in zip(unique, counts):
    bw_flags = bitwise_scheme.get_flags(mask=val)
    for flag in bw_flags:
        print(f"  {flag.label}: {count} vectors ({count / flags.size * 100:.1f}%)")

Generated PIV data: (50, 100)
Flag distribution:
  Interpolated: 24 vectors (0.5%)
  Replaced: 280 vectors (5.6%)


## Integration with PIV Datasets

Flag schemes can be integrated with PIV datasets for complete metadata:

In [5]:
# Create a PIV dataset with flag scheme
from pivmetalib import pivmeta, prov

# Create dataset description
dataset = pivmeta.ImageVelocimetryDataset(
    title="PIV Data with Quality Flags",
    description="Example dataset demonstrating flag usage",
    creator=prov.Person(
        lastName="Demo",
        firstName="User",
        mbox="demo@example.com"
    ),
    hasFlagScheme=simple_scheme
)

# Create distribution with flag information
distribution = pivmeta.ImageVelocimetryDistribution(
    title="PIV velocity field with piv flags",
    description="2D velocity field with PIV Flags",
    mediaType="application/x-hdf",
    hasFlagScheme=simple_scheme,
    numberOfRecords=height * width
)

# Add distribution to dataset
dataset.distribution = [distribution]

print("Created PIV dataset with flag scheme:")
print(f"Dataset: {dataset.title}")
print(f"Flag scheme: {simple_scheme.label}")
print(f"Number of flags: {len(simple_scheme.allowedFlag)}")

# Export to JSON-LD
metadata_ttl = dataset.model_dump_ttl()
print("\nDataset metadata generated successfully!\n")

print(metadata_ttl)

Created PIV dataset with flag scheme:
Dataset: PIV Data with Quality Flags
Flag scheme: Basic PIV Quality Flags
Number of flags: 3

Dataset metadata generated successfully!

@prefix dcat: <http://www.w3.org/ns/dcat#> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix piv: <https://matthiasprobst.github.io/pivmeta#> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<https://example/enumeratedFlagSchemeType> a piv:EnumeratedFlagScheme .

[] a piv:ImageVelocimetryDataset ;
    dcterms:creator [ a prov:Person ;
            foaf:firstName "User" ;
            foaf:lastName "Demo" ;
            foaf:mbox "demo@example.com" ] ;
    dcterms:description "Example dataset demonstrating flag usage" ;
    dcterms:title "PIV Data with Quality Flags" ;
    dcat:distribution [ a piv:ImageVelocimetryDistribution ;
            dcterms:description "2D

## Advanced Usage Patterns

### Custom Flag Schemes

Create specialized flag schemes for different PIV applications:

In [6]:
# Specialized flag scheme for turbulent flow analysis
turbulent_scheme = FlagScheme(
    label="Turbulent Flow Analysis Flags",
    description="Flags for turbulent PIV analysis",
    flagSchemeType=EnumeratedFlagScheme(),
    flags=[
        Flag(mask=0, label="Good", meaning="Valid turbulent vector"),
        Flag(mask=1, label="Spike", meaning="Velocity spike detected"),
        Flag(mask=2, label="Outlier", meaning="Statistical outlier"),
        Flag(mask=3, label="Smoothed", meaning="Vector smoothed by filter"),
        Flag(mask=4, label="NaN", meaning="Not a number value"),
        Flag(mask=5, label="Interpolated", meaning="Gap-filled value"),
    ]
)

# Specialized scheme for boundary layer analysis
boundary_layer_scheme = FlagScheme(
    label="Boundary Layer Analysis Flags",
    description="Flags for boundary layer PIV analysis",
    flagSchemeType=EnumeratedFlagScheme(),
    flags=[
        Flag(mask=0, label="Good", meaning="Valid boundary layer vector"),
        Flag(mask=1, label="WallReflection", meaning="Wall reflection artifact"),
        Flag(mask=2, label="ShadowRegion", meaning="Shadow/low intensity region"),
        Flag(mask=3, label="SurfaceTangent", meaning="Surface-tangent constraint applied"),
        Flag(mask=4, label="Filtered", meaning="Filtered by boundary condition"),
    ]
)

print("Created specialized flag schemes:")
print(f"- Turbulent flow: {len(turbulent_scheme.flags)} flags")
print(f"- Boundary layer: {len(boundary_layer_scheme.flags)} flags")

Created specialized flag schemes:
- Turbulent flow: 6 flags
- Boundary layer: 5 flags
